# Import modules

In [1]:
from IPython.core.display import display, HTML
import numpy as np
import pandas as pd
from textblob import TextBlob
import textstat
from multiprocessing import  Pool

from bs4 import BeautifulSoup
import re
import statistics



# Optional: Strip HTML

In [ ]:
# cols = ['ClapCount_Card', 'ClapCount_Story', 'Company', 'CompanyURL', 'PostID',
#        'PublishedDate', 'ReadingTime', 'ResponseNum_Card', 'ResponseNum_Story',
#        'StoryHTML', 'StoryIndex', 'StoryTitle', 'StoryURL', 'StoryURL_Story',
#        'Tag1', 'Tag2', 'Tag3', 'Tag4', 'Tag5', 'TagSource', 'User', 'UserURL',
#        'VoterCount', 'isPaywall']


cols = ['ClapCount_Card', 'ClapCount_Story', 'Company', 'CompanyURL', 'PostID',
       'PublishedDate', 'ReadingTime', 'ResponseNum_Card', 'ResponseNum_Story',
       'StoryIndex', 'StoryTitle', 'StoryURL', 'StoryURL_Story',
       'Tag1', 'Tag2', 'Tag3', 'Tag4', 'Tag5', 'TagSource', 'User', 'UserURL',
       'VoterCount', 'isPaywall']
import pandas as pd
reader = pd.read_csv('df_story.csv', chunksize=20000)
for chunk in reader:
    chunk.to_csv('df_noHTML.csv', index=False, header=False, mode='a')

    result = chunk
    result['StoryHTML'] = ''

    result.to_csv('df_noHTML.csv', index=False, header=False, mode='a')



In [ ]:
df_noHTML = pd.read_csv("df_noHTML.csv", names=cols)
display(HTML(df_noHTML.head(5).to_html()))

# Import data and filter N/A stories

In [ ]:
df = pd.read_csv("data/df_story_chunk1.csv", nrows=100)
df.head()
df = df[-df['PostID'].isnull()]
df = df[-df['ReadingTime'].isnull()]

print(len(df))


# Load author data

In [2]:
authorStats = pd.read_csv("data/clean/authorStats.csv")

In [ ]:
authorStats.columns

# Load tag data

In [3]:
tagStats = pd.read_csv("data/clean/tagStats.csv")
tagStats1 =  tagStats.copy()
tagStats2 = tagStats.copy()
tagStats3 =  tagStats.copy()
tagStats4 = tagStats.copy()
tagStats5 =  tagStats.copy()
print(tagStats.columns)
tagStats1.columns = ['Tag1', 'Tag1Clap_mean', 'Tag1Clap_median', 'Tag1Clap_count', 'Tag1Clap_sum', 'Tag1Clap_std', 'Tag1Response_mean', 'Tag1Response_median','Tag1Response_count', 'Tag1Response_sum', 'Tag1Response_std', 'Tag1ReadingTime_mean', 'Tag1ReadingTime_median', 'Tag1ReadingTime_count','Tag1ReadingTime_sum', 'Tag1ReadingTime_std']
tagStats2.columns = ['Tag2', 'Tag2Clap_mean', 'Tag2Clap_median', 'Tag2Clap_count', 'Tag2Clap_sum', 'Tag2Clap_std', 'Tag2Response_mean', 'Tag2Response_median','Tag2Response_count', 'Tag2Response_sum', 'Tag2Response_std', 'Tag2ReadingTime_mean', 'Tag2ReadingTime_median', 'Tag2ReadingTime_count','Tag2ReadingTime_sum', 'Tag2ReadingTime_std']
tagStats3.columns = ['Tag3', 'Tag3Clap_mean', 'Tag3Clap_median', 'Tag3Clap_count', 'Tag3Clap_sum', 'Tag3Clap_std', 'Tag3Response_mean', 'Tag3Response_median','Tag3Response_count', 'Tag3Response_sum', 'Tag3Response_std', 'Tag3ReadingTime_mean', 'Tag3ReadingTime_median', 'Tag3ReadingTime_count','Tag3ReadingTime_sum', 'Tag3ReadingTime_std']
tagStats4.columns = ['Tag4', 'Tag4Clap_mean', 'Tag4Clap_median', 'Tag4Clap_count', 'Tag4Clap_sum', 'Tag4Clap_std', 'Tag4Response_mean', 'Tag4Response_median','Tag4Response_count', 'Tag4Response_sum', 'Tag4Response_std', 'Tag4ReadingTime_mean', 'Tag4ReadingTime_median', 'Tag4ReadingTime_count','Tag4ReadingTime_sum', 'Tag4ReadingTime_std']
tagStats5.columns = ['Tag5', 'Tag5Clap_mean', 'Tag5Clap_median', 'Tag5Clap_count', 'Tag5Clap_sum', 'Tag5Clap_std', 'Tag5Response_mean', 'Tag5Response_median','Tag5Response_count', 'Tag5Response_sum', 'Tag5Response_std', 'Tag5ReadingTime_mean', 'Tag5ReadingTime_median', 'Tag5ReadingTime_count','Tag5ReadingTime_sum', 'Tag5ReadingTime_std']


Index(['TagSource', 'TagClap_mean', 'TagClap_median', 'TagClap_count',
       'TagClap_sum', 'TagClap_std', 'TagResponse_mean', 'TagResponse_median',
       'TagResponse_count', 'TagResponse_sum', 'TagResponse_std',
       'TagReadingTime_mean', 'TagReadingTime_median', 'TagReadingTime_count',
       'TagReadingTime_sum', 'TagReadingTime_std'],
      dtype='object')


In [ ]:
tagStats.columns

# Load publication data

In [4]:
publicationStats = pd.read_csv("data/clean/publicationStats.csv")
publicationStats.head()
print(publicationStats.isna().any())

Company                          False
PublicationClap_mean             False
PublicationClap_median           False
PublicationClap_count            False
PublicationClap_sum              False
PublicationReadingTime_mean      False
PublicationReadingTime_median    False
PublicationReadingTime_count     False
PublicationReadingTime_sum       False
PublicationVoter_mean            False
PublicationVoter_median          False
PublicationVoter_count           False
PublicationVoter_sum             False
PublicationisPaywall_mean        False
PublicationisPaywall_median      False
PublicationisPaywall_count       False
PublicationisPaywall_sum         False
dtype: bool


# Extract features from HTML

In [ ]:
%%time
print('hello')

# Generate feature extracted data

In [ ]:
%%time
# pd.options.display.max_colwidth = 10000
filepath = 'data/features/df_story_features_chunk2.csv'
# Remove csv if exists
import os
if os.path.exists(filepath):
  os.remove(filepath)

import extractFeaturesFunction

count = 0
header = True

# numRows = 10000
chunkSize =50000

processedRows = 0
chunkRows = 0
 # Read chunks
# for data in pd.read_csv('data/raw/OneDrive - University Of Waterloo/df_story_chunk2.csv',encoding = 'ISO-8859-1', nrows=numRows, chunksize  = chunkSize, low_memory=False):
 # Read entire data
for data in pd.read_csv('data/raw/OneDrive - University Of Waterloo/df_story_chunk2.csv',encoding = 'ISO-8859-1', chunksize  = chunkSize, low_memory=False):

    count += 1                          # counting the number of chunks
    lastlen = len(data)                 # finding the length of last chunk

    # Filter NaN PostID
    data = data[-data['PostID'].isnull()]      
    
    # Filter NaN ResponseTime
    data = data[-data['ReadingTime'].isnull()] 

    # Filter stories posted after April 2020
    data['PublishedDate'] = pd.to_datetime(data['PublishedDate']).dt.date
    data = data[data['PublishedDate'] < pd.to_datetime("2020-04-01")]

    
    # Convert responsetime "X min read" to X as int
    data['ReadingTime'] = data['ReadingTime'].str.extract('(\d+)', expand=False).astype(int) 
    
    # Get number of tags used
    data['TagNum'] = data[['Tag1','Tag2','Tag3','Tag4', 'Tag5']].notnull().sum(axis=1)

    # Boolean, if article belongs to publication
    data['isPublication'] = data['Company'].notnull()
    
    # Extract features  from HTML
    data[['Text', 'SentimentPolarity', 'SentimentSubjectivity', 'WordNum', "TextSyllableNum",
          "TextLexiconNum", "TextSentenceNum", "ReadabilityFleschEase", "ReadabilitySMOG", "ReadabilityFleschKincaid", "ReadabilityColemanLiau", "ReadabilityARI", "ReadabilityDaleChall", "ReadabilityDifficultWordsList", "ReadabilityDifficultWordsNum", "ReadabilityLinsearWriteFormula", "ReadabilityGunningFog", "ReadabilityReadingTime", "ReadabilityConsensus", 
          'HasFeaturedImage','CodeInlineRaw', 'CodeInlineNum', 
          'CodeBlockRaw', 'CodeBlockNum', "CodeBlockLengthList", "CodeBlockLengthSum", "CodeBlockLengthMedian", "CodeBlockLengthMean", "CodeBlockLengthStd", "CodeBlockLengthMin", "CodeBlockLengthMax",
          'ListOlNum', 'ListOlLength', 'ListOlSum', 'ListOlMedian', 'ListOlMean', 'ListOlStd', "ListOlMin", 'ListOlMax',
          'ListUlNum', 'ListUlLength', 'ListUlSum', 'ListUlMedian', 'ListUlMean', 'ListUlStd', "ListUlMin", 'ListUlMax',
          'ImgNum', 'LinkURLList', 'LinkNum', 'HLightTextList', 'HlightNum', 'ParagraphNum', "ItalicNum", "BoldNum"]] = pd.DataFrame(extractFeaturesFunction.parallelize_on_rows(data, extractFeaturesFunction.extract_features_from_html_multiproc).tolist())

#     print(type(mypd))
    # Join author data
    data = pd.merge(data, authorStats, on="User")   
    
    # Join publication stats
    
    # PublicationClapCount: number of articles that the publication contains
    data = pd.merge(data, tagStats1, on="Tag1", how='left')
    data = pd.merge(data, tagStats2, on="Tag2", how='left')
    data = pd.merge(data, tagStats3, on="Tag3", how='left')
    data = pd.merge(data, tagStats4, on="Tag4", how='left')
    data = pd.merge(data, tagStats5, on="Tag5", how='left')

    # Join publication data
    data = pd.merge(data, publicationStats, on="Company", how='left').fillna(0)
    
    # Sum tag uses for all tags
    
    # TagUseSum: total number of times that Tag1,2,3,4,5 have been used
    # TagUseMean: mean usage number of Tag1,2,3,4,5
    # TagUseMedian, median usage number of Tag1,2,3,4,5
    
    # TagClapSum: total number of claps of articles in Tag1,2,3,4,5    
    data['TagUseSum'] = data['Tag1Clap_count'].fillna(0) + data['Tag2Clap_count'].fillna(0) + data['Tag3Clap_count'].fillna(0) + data['Tag4Clap_count'].fillna(0) + data['Tag5Clap_count'].fillna(0)
    data['TagUseMean'] = data['TagUseSum'].fillna(0)/data['TagNum'].fillna(1)
    data['TagUseMedian'] = data[['Tag1Clap_count', 'Tag2Clap_count', 'Tag3Clap_count', 'Tag4Clap_count', 'Tag5Clap_count']].median(axis=1)
    
    data['TagClapSum']  = data['Tag1Clap_sum'].fillna(0) + data['Tag2Clap_sum'].fillna(0) + data['Tag3Clap_sum'].fillna(0) + data['Tag4Clap_sum'].fillna(0) + data['Tag5Clap_sum'].fillna(0)
    data['TagClapMean'] = data['TagClapSum'].fillna(0)/data['TagNum'].fillna(1)
    data['TagClapMedian'] = data[['Tag1Clap_sum', 'Tag2Clap_sum', 'Tag3Clap_sum', 'Tag4Clap_sum', 'Tag5Clap_sum']].median(axis=1)

    data['TagUseMean'].fillna(0, inplace=True)
    data['TagUseMedian'].fillna(0, inplace=True)
    data['TagClapMean'].fillna(0, inplace=True)
    data['TagClapMedian'].fillna(0, inplace=True)
    
    # Delete raw html
    del data['StoryHTML']
    
    processedRows += len(data)
    chunkRows += chunkSize
    print("Processed", chunkRows, "rows. Processed ", processedRows, "clean rows.")
    
    data.to_csv(filepath, header=header, mode="a")
    header = False
datalength = (count*chunkSize + lastlen - chunkSize) # length of total file
print(datalength, "rows read", processedRows, "rows clean")

Processed 50000 rows. Processed  40141 clean rows
Processed 100000 rows. Processed  85131 clean rows
Processed 150000 rows. Processed  126052 clean rows
Processed 200000 rows. Processed  164925 clean rows
Processed 250000 rows. Processed  207255 clean rows
Processed 300000 rows. Processed  251102 clean rows
Processed 350000 rows. Processed  292404 clean rows
Processed 400000 rows. Processed  337356 clean rows
Processed 450000 rows. Processed  373148 clean rows


In [ ]:
test = pd.read_csv("data/features/df_story_features_chunk2.csv")
display(HTML(test.sample(2).to_html()))

In [ ]:
# display(HTML(test.merge(authorStats[['User', 'ClapCount_Story_count']], on="User").sample(5).to_html()))


In [ ]:
test_html = df.iloc[547].StoryHTML
testsoup = BeautifulSoup(test_html, 'lxml')
print(testsoup.find_all("figure", {'class':re.compile("/.*[^image]$/")}))

In [ ]:
# display(HTML(df.head(5).to_html()))
pd.set_option('display.max_colwidth', -1)

print(df[df['CodeNum']>0].head(100)['StoryURL'].to_string())



In [ ]:
import matplotlib.pyplot as plt
print(df[df['CodeBlockLengthMean']>0]['CodeBlockLengthMean'].mean())
df['CodeBlockLengthMean'].hist(bins=60)
plt.show()